# OpenCascade Geometry Processing

"OpenCascade Technology" (OCCT) is a collection of 3D CAD libraries written in C++ and released under LGPL. It is "designed for rapid production of sophisticated domain-specific CAD/CAM/CAE applications" and provides the basis for FreeCAD and several open-source BIM projects. Python bindings are provided through at least two projects, [pyOCCT](https://github.com/trelau/pyOCCT) (\~80 github stars, linux/win only) and [pythonocc-core](https://github.com/tpaviot/pythonocc-core) (\~700 github stars, cross-platform including MacOS).

It is a large, complicated, old (begun in 1999) project that isn't widely used and therefore doesn't have extensive informal documentation (tutorials, examples and forums). Using OCCT in Python has the further complication of needing to translate between C++ and Python syntax. While documentation for pythonocc-core is available, it is not hosted online in an accessible format. There is, however, a repository containing [examples](https://github.com/tpaviot/pythonocc-demos) in Python.

The high-level [guide](https://dev.opencascade.org/doc/overview/html/), linked as "full online documentation" on the project website, is extremely helpful for understanding OCCT on a conceptual level, while the [c++ api docs](https://dev.opencascade.org/doc/refman/html/), confusingly linked as "reference manual", provide low-level details on class inheritance, methods and properties. See also the [forums](https://dev.opencascade.org/forums).

Other resources:

- http://opencascade.wikidot.com
- https://opencascade.blogspot.com/2009/02/topology-and-geometry-in-open-cascade_12.html

CGAL, which provides the basis for SFCGAL support in PostGIS and underpins OpenSCAD is a potential alternative, although it is not designed as a full-featured BRep modeling tool and focuses more on meshes. Bindings for CGAL can be generated for other languages through SWIG.

In [1]:
import sys
import re
import math
import copy

In [2]:
import shapely.wkt

OCCT is organized into seven modules, imports we'll use here are listed below:

In [3]:
# --- FOUNDATION CLASSES ---

from OCC.Core.TColgp import TColgp_HArray1OfPnt, TColgp_Array1OfPnt2d, TColgp_Array1OfPnt
from OCC.Core.TColgp import TColgp_SequenceOfPnt
from OCC.Core.GeomAbs import GeomAbs_C2


# --- MODELING DATA ---

from OCC.Core.gp import gp_Pnt, gp_Pnt2d, gp_Vec, gp_Dir, gp_Ax1, gp_Ax2, gp_Pln, gp_Trsf # primitives
from OCC.Core.TopLoc import TopLoc_Location
from OCC.Core.Geom2d import (
    Geom2d_Curve, 
    Geom2d_OffsetCurve,
)
from OCC.Core.Geom import Geom_Curve, Geom_OffsetCurve
from OCC.Core.gce import gce_MakePln, gce_MakeLin # direct construction
from OCC.Core.TopoDS import TopoDS_Iterator
from OCC.Core.BRepAdaptor import BRepAdaptor_CompCurve
from OCC.Core.Geom2dAdaptor import Geom2dAdaptor_Curve

# --- MODELING ALGORITHMS ---

# high-level modeling routines (boolean ops, construction of primitives, kinematics)
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakePolygon, BRepBuilderAPI_MakeFace, BRepBuilderAPI_MakeShell, BRepBuilderAPI_MakeSolid
from OCC.Core.BRepOffsetAPI import BRepOffsetAPI_ThruSections

# low-level mathematical support functions (intersections, projections, interpolations)
from OCC.Core.Geom2dAPI import (
    Geom2dAPI_Interpolate, 
    Geom2dAPI_PointsToBSpline,
    Geom2dAPI_InterCurveCurve,
    Geom2dAPI_ProjectPointOnCurve,
)

from OCC.Core.GeomAPI import (
    geomapi_To3d,
    geomapi_To2d,
    GeomAPI_PointsToBSpline,
    GeomAPI_Interpolate, 
    GeomAPI_PointsToBSplineSurface,
)
from OCC.Core.GeomLib import geomlib


### visualization


### data exchange


### application framework

from OCC.Core.BOPAlgo import BOPAlgo_MakeConnected, BOPAlgo_BuilderSolid
from OCC.Core.BRepAlgoAPI import (
    BRepAlgoAPI_Fuse, 
    BRepAlgoAPI_Common,
    BRepAlgoAPI_Cut,
)
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakePrism
from OCC.Core.ShapeUpgrade import ShapeUpgrade_UnifySameDomain
from OCC.Core.TopTools import TopTools_ListOfShape

### pythonocc-core also gives us some extra stuff

from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer


In [4]:
# add udtools core modules to path for testing
module_path = '../python/'
if module_path not in sys.path:
    sys.path.append(module_path)
    
from geometry.brep import loft_wires, upgrade
from geometry.conversion import (
    wkt_to_occ, 
    wire_to_face, 
    curve_to_wire, 
    coords_to_array, 
    coords_to_curve, 
    curve_to_face, 
    curve_to_wire,
    coords_to_polygon,
)
from geometry.curve import (
    fit_offset_to_bounds,
    offset,
    lift,
)
from geometry.reference import (
    world_origin,
    world_pos_z,
    world_xy,
)
from geometry.zoning import build_yard_cutter

Input geometry will mostly come from PostGIS in the form of [well-known text](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) and potentially later well-known binary. Here's some data for testing:

In [5]:
test_edge = 'LINESTRING(-46.22362966486253 -23.632720552763203,-59.2367360109929 -71.32453354485915)'
test_bounds = 'POLYGON((-46.22362966486253 -23.632720552763203,-59.2367360109929 -71.32453354485915,-160.80903549143113 -43.799661718687275,-154.23228616663255 -19.696363531431416,-147.7965995783452 3.889560617128154,-141.0705573076848 28.540102876344463,-134.68245048471726 51.951466001191875,-33.1094805712346 24.42985526434495,-46.22362966486253 -23.632720552763203))'

Geometry primitives, provided by the [modeling data](https://dev.opencascade.org/doc/overview/html/index.html#intro_overview_moddata) module and prefixed as `gp_*` provide the basis for all geometric objects in OCCT. Boundary representations (BReps) are represented "as an aggregation of geometry within topology" where **geometry** describes a shape mathematically (curves, surfaces) and **topology** provides a data structure.

In [6]:
point = (1.0, 1.0, 1.0)
occ_point = gp_Pnt(*point)
occ_point

<class 'gp_Pnt'>

The `gce` package provides tools for direct construction of certain primitives, like circles, lines etc.

In [7]:
points = [None, (1.0, 1.0, 1.0), (1.0, 0.0, 0.0)]
occ_line = gce_MakeLin(gp_Pnt(*points[1]), gp_Pnt(*points[2]))
occ_line.Value()

<class 'gp_Lin'>

Compound shapes require OCCT collections as inputs when they're being created. These can be fixed (Array) or variable width (Sequence). Note that in OCCT, arrays are 1-indexed by convention.

In [8]:
points = [
  None, 
  (0.0, 1.0, 1.0), 
  (1.0, 0.0, 0.0), 
  (2.0, 0.0, 0.0)
]

occ_sequence = TColgp_SequenceOfPnt()

for pt in points[1:]:
  occ_pt = gp_Pnt(*pt)
  occ_sequence.Append(occ_pt)

occ_array = TColgp_Array1OfPnt(1, 2)
occ_array.SetValue(1, gp_Pnt(*points[1]))
occ_array.SetValue(2, gp_Pnt(*points[2]))

occ_array_2 = TColgp_Array1OfPnt(1, 4)
occ_array_2.SetValue(1, gp_Pnt(*points[1]))
occ_array_2.SetValue(2, gp_Pnt(*points[2]))
occ_array_2.SetValue(3, gp_Pnt(*points[3]))
occ_array_2.SetValue(4, gp_Pnt(*points[1]))

occ_harray = TColgp_HArray1OfPnt(occ_array) # can make HArray from Array
#occ_harrayfromseq = TColgp_HArray1OfPnt(occ_sequence) # but unclear if/how possible w sequences

These can then be used to create curves (`Geom_BSplineCurve`). See the Wikipedia article on [splines](https://en.wikipedia.org/wiki/Spline_%28mathematics%29) for a detailed explanation of how this is represented mathematically.

In [9]:
# open curve, method 1
interp = GeomAPI_Interpolate(occ_harray, False, 0.01)
interp.Perform()
crv = interp.Curve()

# closed curve
interp_closed = GeomAPI_Interpolate(occ_harray, True, 0.5)
interp_closed.Perform()
crv_closed = interp_closed.Curve()

# open curve, method 2
crv2 = GeomAPI_PointsToBSpline(occ_array).Curve()

# closed curve, method 2
# will be closed if the first and last elements in the array are the same
# will have a higher degree automatically unless a linear spline is specified
# with the degree min/max parameters
crv_closed_2 = GeomAPI_PointsToBSpline(occ_array_2, 1, 1).Curve()

for i, c in enumerate([crv, crv_closed, crv2, crv_closed_2]):
  print(f'Curve {i + 1} is {"closed" if c.IsClosed() else "open"} and has a degree of {c.Degree()}')

Curve 1 is open and has a degree of 1
Curve 2 is closed and has a degree of 1
Curve 3 is open and has a degree of 1
Curve 4 is closed and has a degree of 1


In [10]:
# get point on and tangent vector at param U of 0.5
# p = gp_Pnt()
# v = gp_Vec()
# print(v.X())
# edge_curve.D1(0.5, p, v)

In [11]:
# or surfaces from array2
# srf = GeomAPI_PointsToBSplineSurface(occ_2array, 1, 1, GeomAbs_C2, 0.1)

In [12]:
# lines are infinitely long
# c2_lin = gce_MakeLin(gp_Pnt(*c2[0]), gp_Pnt(*c2[1])).Value()
# c2_lin_2d = geomapi_To2d(c2_lin, plane_2d)

Topology objects like edges, wires, polygons, faces, etc. can also be built up from a sequence of points. Alternately, they can be converted from geometry objects. Curves can be converted to edges with `BRepBuilderAPI_MakeEdge`, and combined together into a wire with `BrepBuilderAPI_MakeWire`.

In [13]:
wire = wkt_to_occ(test_edge)
bounds = wkt_to_occ(test_bounds)
boundsface = wire_to_face(bounds)

In [14]:
# from a curve, you can then build up topological objects like so:
edge = BRepBuilderAPI_MakeEdge(crv_closed_2).Shape()
wire = BRepBuilderAPI_MakeWire(edge).Shape()
face = BRepBuilderAPI_MakeFace(wire).Shape()
face

<class 'TopoDS_Face'>

Wires can be used as curves with an adaptor, such as `BRepAdaptor_CompCurve`.

In [15]:
adaptor = BRepAdaptor_CompCurve(bounds)
adaptor.Value(0.5)
adaptor.IsClosed()

True

Using modeling APIs, more complex shapes can be built up from geometry and topology objects.

In [16]:
# create a linear swept topology, which OCCT calls a prism
# and Rhino calls an extrusion
distance = 10
vector = gp_Vec(0, 0, distance)
extrusion = BRepPrimAPI_MakePrism(face, vector).Shape()

# if a direction is provided instead of a vector
# the extrusion should be infinite
# see "Finite, infinite, and semi-infinite prisms" in the guide
direction = gp_Dir(0, 0, 1)
extrusion_infinite = BRepPrimAPI_MakePrism(face, direction).Shape()

# note: shapes of infinite prisms don't work with the Jupyter renderer

In [17]:
# offset a curve, then make it an edge for display
offset_result = Geom_OffsetCurve(crv2, 1.0, gp_Dir(), False)
offset_edge = BRepBuilderAPI_MakeEdge(offset_result).Shape()

In [18]:
offset_2 = Geom_OffsetCurve(offset_result, 2.0, gp_Dir(0,0,1), False)

In [19]:
# loft some curves
loft = BRepOffsetAPI_ThruSections(True, True, 0.1)
loft.AddWire(curve_to_wire(crv2))
loft.AddWire(curve_to_wire(offset_result))
loft.AddWire(curve_to_wire(offset_2))
#loft.AddWire(curve_to_wire(crv2)) # add last one as first to close
loft_result = loft.Shape()
# how to make solid? (cap holes?)

Boolean operations are accomplished through the OCCT BRepAlgoAPI, which provides FUSE (union), COMMON (intersection) and CUT (difference) operators, explained [here](https://dev.opencascade.org/doc/overview/html/specification__boolean_operations.html). These can all work with infinite and semi-infinite prisms.

In [21]:
# define "zoning lot" bounds as OCC curve
coords = [
    (0,0,0),
    (1,0,0),
    (1.1,1,0),
    (0,1,0),
    (0,0,0)
]

bounds = coords_to_curve(coords)

In [22]:
# use the same coordinate sequence to make an extrusion to use as the basis of the envelope

polygon = coords_to_polygon(coords)
face = BRepBuilderAPI_MakeFace(polygon).Shape()
distance = 3
vector = gp_Vec(0, 0, distance)
extrusion = BRepPrimAPI_MakePrism(face, vector).Shape()

In [23]:
# define a planar cutter
# infinite in the positive z direction only
# use this to set lower limit for parking garage,
# maximum height
# sky exposure planes etc

cutter_coords = [
    (0,0,2),
    (1,0,2),
    (1.1,1,3),
    (0,1,3),
    (0,0,2)
]

polygon = coords_to_polygon(cutter_coords)
face = BRepBuilderAPI_MakeFace(polygon).Shape()
direction = gp_Dir(0, 0, 1)
sloped_cutter = BRepPrimAPI_MakePrism(
    face, 
    direction, 
    False
).Shape()

In [24]:
# define one lot edge and setback, make cutter

c1 = [
  (0,0,0), # start
  (1,0,0), # end
]

c1_dims = [
  [0.2, 0], # 4' yard
  [0.2, 0.5], # 5' base height
  [0.4, 0.5], # +1' setback
  [0.4, 0.7], # 7' roof ht
]

c1 = coords_to_curve(c1)
loft_1 = build_yard_cutter(c1, bounds, c1_dims)


curve 1:
...set back by 0.2
...elevated by 0
curve 2:
...set back by 0.2
...elevated by 0.5
curve 3:
...set back by 0.4
...elevated by 0.5
curve 4:
...set back by 0.4
...elevated by 0.7


In [25]:
# define a second lot edge and setback, make cutter

c2 = [
  (1.1,1,0),
  (0,1,0),
]

c2_dims = [
  [0.1, 0], # 4' yard
  [0.1, 0.5], # 5' base height
  [0.2, 0.5], # +1' setback
  [0.2, 0.7], # 7' roof ht
]

c2 = coords_to_curve(c2)
loft_2 = build_yard_cutter(c2, bounds, c2_dims)

curve 1:
...set back by 0.1
...elevated by 0
curve 2:
...set back by 0.1
...elevated by 0.5
curve 3:
...set back by 0.2
...elevated by 0.5
curve 4:
...set back by 0.2
...elevated by 0.7


In [26]:
# one more...

c3 = [
  (0,1,0),
  (0,0,0),
]

c3_dims = [
  [0.1, 0],
  [0.1, 1],
]

c3 = coords_to_curve(c3)
loft_3 = build_yard_cutter(c3, bounds, c3_dims)


curve 1:
...set back by 0.1
...elevated by 0
curve 2:
...set back by 0.1
...elevated by 1


In [27]:
# iterate through cutters to trim down initial extrusion
result = extrusion

for c in [sloped_cutter, loft_1, loft_2, loft_3]:
  result = BRepAlgoAPI_Cut(result, c).Shape()

result = upgrade(result)


In [28]:
# set up renderer
renderer = JupyterRenderer()

In [29]:
renderer.DisplayShape(result, render_edges=True, topo_level="default", shape_color="#abdda4", update=True)

## Questions

- What is the difference between Array and HArray?
- Can sequences be cast to arrays?
- To confirm, do geometry objects need to be put into a topology structure before they can be rendered? Exported?
